<a href="https://colab.research.google.com/github/NikolaevMikhailRoma/Mesoyakha_Project/blob/main/Convertor_csv_to_gtm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Конвертор из .csv в .gtm

Сделано по прихоти заказчика (прогер из криолаба говорил что конвертор невозможен)

Файлы формата .gtm можно загрузить в среду колаба или через гуглдиск

Примечания:
1. Возможно небходимо добавить столбцы в .csv с 12,0 , 14,0 , 15,0 отметками (пустые), не помню точно

# 1 Вариант. Конвертируем в среду колаба
Загружаем файлы с компьютера: 
*   На левой вкладке для инструментов нажимаем на иконку с папкой (последняя иконка с заначком - папкой)
  *   Загружаем .csv файлы (первая кнопка слева в открывшемся окне (иконка с листом и стрелочкой вверх))
  *   Выполняем следующий блок кода
  *   Обновляем вкладку с файлами (средняя иконка слева в открытом окне, правее иконки загрузки)
  *   Скачиваем .gtm-ки через значкек с троеточием (появляется если наверсти мышку на файл

In [9]:
import io
import pickle
import pandas as pd
import sys
import os
import pandas as pd
from importlib import reload
import struct
# import lib_conv_csv_gtm as lib
import numpy as np

list_of_zero_level_csv_path = os.listdir() # 
list_of_csv_path = []
for i1 in list_of_zero_level_csv_path:
  if '.csv' in i1:
    list_of_csv_path.append(i1)
for csv_file in list_of_csv_path:        
        # открываем и читаем датафрейм от csv файла
        # df_of_csv_file = pd.read_csv(csv_file, encoding='cp1251', sep = ';', decimal=',') #правка 17.20.2021
        df_of_csv_file = pd.read_csv(csv_file, sep = ';', decimal=',')
        df_of_csv_file.rename(columns ={'объект' : 'скважина'}, inplace = True) # переименовываем
        actual_columns = ['0,0', '0,5', '1,0', '1,5', '2,0', '2,5', '3,0', '3,5', '4,0', 
                      '4,5', '5,0', '6,0', '7,0', '8,0', '9,0', '10,0', '12,0', '14,0', '15,0', 'дата время', 'коса', 'скважина']
        
        #добавляем новые индексы, чтобы можно было использовать 15м косы
        df_of_csv_file = df_of_csv_file.reindex(columns = actual_columns)
        df_of_csv_file = df_of_csv_file[actual_columns] # на всякий
        df_of_csv_file['дата время'] = pd.to_datetime(df_of_csv_file['дата время'], dayfirst=True)

        #GTM FILE
        path = csv_file[:-4] + '.gtm' # обозначаем файл gtm на основе файла csv
        ### open and create gpm file
        file = open(path, 'wb')
        file.write(b'GTM0')
        
#         continue #
        for i in range (0, len(df_of_csv_file)): 
            count_censors = 16 # кол-во датчиков температур в стандартной косе

            # первый байт скорее всего зависит от кол-ва символов в строке, \x63 для 3 в строке
            # блок отпеделения размера записи
            name_object = str(df_of_csv_file.iloc[i]['скважина'])
            len_of_well_name = len(name_object)

            ###
            more_sensors = 3
            # проверяем размер косы, если есть измерение на 12 метрах то считаем запись как для 15-м косы
            if np.isnan(df_of_csv_file.iloc[i]['12,0']): 
                more_sensors = 0
            count_censors = count_censors+more_sensors
            file.write(struct.pack('>h', 96+len_of_well_name+more_sensors*5)) # длинна записи, хз зачем нужна такая информация
            
            ###
            file.write(b'\x54\x30\x32') # добавляет T02 не знаю что это
            file.write(struct.pack('b', count_censors)) # кол-во датчиков в косе
            
            ###########################
            ##### блок времени и даты
            ##### блок написания времени
            minute = df_of_csv_file.iloc[i]['дата время'].minute
            second = df_of_csv_file.iloc[i]['дата время'].second
            file.write(struct.pack('B', second + (minute%8)*32)) # cекунды и остаток минут деленные на 8
            hour = df_of_csv_file.iloc[i]['дата время'].hour
            file.write(struct.pack('B', hour*8 + (minute//8))) # час и минута деленная на 8

            ##### блок написания дня и месяца
            month = df_of_csv_file.iloc[i]['дата время'].month
            day = df_of_csv_file.iloc[i]['дата время'].day  
            file.write(struct.pack('B', day+32*(month%8))) # дата от 01.01.1980
            ## Блок написания года 
            year = df_of_csv_file.iloc[i]['дата время'].year-1980
            polovina_goda=0
            if month >=8:
                polovina_goda=1
            file.write(struct.pack('B', year*2+polovina_goda))
            ###########################
            ###########################

            ### блок записи номера косы
            file.write(struct.pack('>h', df_of_csv_file.iloc[i]['коса']))
            ### блок добавления превышения уровня нуля
            file.write(b'\x00\x00') # превышение уровня 0!!!, у нас его нет

            ### блок названия объекта
            file.write(struct.pack('B', len_of_well_name+1)) # длинна имени объекта +1
            file.write(bytes(name_object, 'utf-8'))
            file.write(b'\x00')
            ###

            ###блок написания значений температур (самое главное)
            actual_columns = ['0,0', '0,5', '1,0', '1,5', '2,0', '2,5', '3,0', '3,5', '4,0', 
                              '4,5', '5,0', '6,0', '7,0', '8,0', '9,0', '10,0', '12,0', '14,0', '15,0']
            offset_sensor = [0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 10, 10, 10, 10, 10, 20, 20, 10]
            for index, j in enumerate(actual_columns):
                if str(df_of_csv_file.iloc[i][j]) != 'nan': # проверка на nan
                    current_temperature_number = float(df_of_csv_file.iloc[i][j])
                    integer_part = int(abs(current_temperature_number)//1)*100
                    decimal_part = int(round(abs(current_temperature_number)*100, 0)%100)
                    number = integer_part+decimal_part
                    if current_temperature_number<0:
                        number = number*(-1)
                    file.write(struct.pack('>h', number))
                    file.write(b'\xfe\n') #пишем степень и знак цельсия
                    file.write(struct.pack('>B', offset_sensor[index]))

            ### последний байт, длинна строки, опять, зачееем???
            file.write(struct.pack('>h', 96+len_of_well_name+more_sensors*5))
        file.close()
#     print (list_of_csv_path)
print('end')


end


In [8]:
pd.read_csv(csv_file, sep = ';', decimal=',')

,дата время,объект,коса,"0,0","0,5","1,0","1,5","2,0","2,5","3,0","3,5","4,0","4,5","5,0","6,0","7,0","8,0","9,0","10,0","12,0","14,0","15,0"
0,11.10.2021 10:33,1021,20417,0.01,0.69,-0.89,1.04,-0.62,-0.90,-0.75,-1.55,-0.80,-0.54,-1.22,-1.34,-0.44,-0.29,-1.35,-0.38,NaN,NaN,NaN
1,11.10.2021 10:48,1231,20435,3.13,4.56,3.84,2.10,3.91,2.23,2.59,3.13,3.05,2.95,4.68,3.91,2.87,2.12,3.49,4.21,2.03,3.27,3.96
2,11.10.2021 10:52,1161,21348,1.65,-1.89,-1.94,-1.64,-1.99,-1.63,-1.96,-2.08,-0.17,-1.97,-1.95,-2.01,-1.95,-1.35,0.56,-1.77,NaN,NaN,NaN
3,11.10.2021 11:00,1072,20437,2.12,1.42,4.77,1.53,1.99,1.69,1.87,3.96,1.98,1.46,2.50,2.25,2.16,2.72,4.34,1.82,2.95,3.59,3.97
4,11.10.2021 11:12,891,20419,1.36,0.49,0.66,1.42,0.29,1.84,-0.24,-0.28,-0.24,-0.35,0.52,-0.08,-0.13,-0.22,0.03,0.40,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,14.10.2021 14:04,1451,20423,0.08,0.22,0.20,0.09,0.23,0.29,-0.10,-0.22,-0.39,-0.54,-0.61,-0.93,-1.14,-1.30,-1.47,-1.67,NaN,NaN,NaN
118,14.10.2021 14:50,12721,20437,0.21,0.92,1.15,0.97,0.53,0.03,-0.10,-0.27,-0.48,-0.51,-0.61,-0.68,-0.76,-0.89,-0.93,-0.98,-0.98,-1.09,-1.16
119,14.10.2021 15:01,241,21357,0.03,0.28,0.17,0.02,-0.17,-0.27,-0.29,-0.29,-0.30,-0.39,-0.35,-0.42,-0.48,-0.49,-0.59,-0.55,NaN,NaN,NaN
120,14.10.2021 15:02,242,21355,-0.01,0.14,0.28,0.16,-0.06,-0.21,-0.23,-0.33,-0.30,-0.36,-0.42,-0.48,-0.55,-0.61,-0.68,-0.72,NaN,NaN,NaN


#2 Вариант: 
1. Загружаем через гугл диск: (выполняем код ячейкой ниже и авторизуемся)

Здесь используем двухуровневая папка (т.е в папке "For_convertor" находятся папки с наименованием объектов (ПСП, Напорник и т.д.) в которых находятся .csv файлы. Нужно обозначить переменную zero_level_csv_path
2. После того как пройдет первый блок, запустить второй. .gtm-ки будут рядом с .csv файлами 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

zero_level_csv_path = "drive/MyDrive/WORK/Месояха/For_Convertor/" # например, нужно дополнительно проверить
list_of_zero_level_csv_path = os.listdir(zero_level_csv_path) # 
list_of_zero_level_csv_path

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['myplot.png',
 'Untitled.ipynb',
 'МЕССОЯХА ТЕРМОМЕТРИЯ ИТОГ ПОЛЕ',
 'Том 12',
 'Том 8',
 'Том 11',
 'На всякий',
 'Том 10',
 '.ipynb_checkpoints',
 'Mesoyaha_passports.ipynb']

In [ ]:
### Здесь пример кода если делать работу скочно
list_of_first_level_csv_path = []
for i1 in list_of_zero_level_csv_path:
    list_of_first_level_csv_path.append(zero_level_csv_path + i1 + '/')
###
# print(list_of_first_level_csv_path)
###
for i1 in list_of_first_level_csv_path:
    
    list_of_end_level_csv_files = []
    list_of_end_level_csv_files = os.listdir(i1)
    
    ### получаем обрабатываемые пути файлов csv
    list_of_csv_path = []
    for i2 in list_of_end_level_csv_files:
        if '.csv' in i2:
            list_of_csv_path.append(i1+i2)
    
    ### начинаем обработку файлов csv и gtm   
    for csv_file in list_of_csv_path:        
        # открываем и читаем датафрейм от csv файла
        df_of_csv_file = pd.read_csv(csv_file, encoding='cp1251', sep = ';', decimal=',')
        df_of_csv_file.rename(columns ={'объект' : 'скважина'}, inplace = True) # переименовываем
        actual_columns = ['0,0', '0,5', '1,0', '1,5', '2,0', '2,5', '3,0', '3,5', '4,0', 
                      '4,5', '5,0', '6,0', '7,0', '8,0', '9,0', '10,0', '12,0', '14,0', '15,0', 'дата время', 'коса', 'скважина']
        #добавляем новые индексы, чтобы использовать 15м косы
        df_of_csv_file = df_of_csv_file.reindex(columns = actual_columns)
        df_of_csv_file = df_of_csv_file[actual_columns] # на всякий
        df_of_csv_file['дата время'] = pd.to_datetime(df_of_csv_file['дата время'], dayfirst=True)

        #GTM FILE
        path = csv_file[:-4] + '.gtm' # обозначаем файл gtm на основе файла csv
        ### open and create gpm file
        file = open(path, 'wb')
        file.write(b'GTM0')
        
#         continue #
        for i in range (0, len(df_of_csv_file)): # !подставить len(df_of_raw_data_files) вместо 5
        #     wtite_to_gtm_file(path = path_of_gtm_file)
            count_censors = 16
#             file = open(path, 'ab') #

            # первый байт скорее зависит от кол-ва символов в строке, \x63 для 3 в строке
            # блок размера записи
            name_object = str(df_of_csv_file.iloc[i]['скважина'])
            len_of_well_name = len(name_object)

            #
            more_sensors = 3
            if np.isnan(df_of_csv_file.iloc[i]['12,0']): #какой пиздец и костыли
                more_sensors = 0
            count_censors = count_censors+more_sensors
            file.write(struct.pack('>h', 96+len_of_well_name+more_sensors*5)) #длинна записи, хз как считается 
            ###


            file.write(b'\x54\x30\x32') # добавляет T02 не знаю что это

            file.write(struct.pack('b', count_censors)) # кол-во датчиков в косе
            ###
# #########################
#             file.write(b'\x00') # cекунды, по умолчанию 0, если больше или равно \x1e то обнуляет
#             ###
#             ###
#             file.write(b'\x00') # минуты*8  хзхзхзхзхз
#             ###
#             ###
#             month = df_of_csv_file.iloc[i]['дата время'].month
            
#             day = df_of_csv_file.iloc[i]['дата время'].day  
#             file.write(struct.pack('B', day+32*month)) # дата от 01.01.1980
#             ###
            
#             ##### Блок написания года
            
#             year = df_of_csv_file.iloc[i]['дата время'].year-1980
#             file.write(struct.pack('B', year*2))
#         #     file.write(b'\x00') # год + 1980, значения по пол года (ну не совсем так)
#             ####
#         #     struct.pack('>h',  df_of_raw_data_files.iloc[i]['скважина'])
#         ###########################
        
            ####################### блок времени и даты
            ##### блок написания времени
            minute = df_of_csv_file.iloc[i]['дата время'].minute
            second = df_of_csv_file.iloc[i]['дата время'].second
            file.write(struct.pack('B', second + (minute%8)*32)) # cекунды и остаток минут деленные на 8
            hour = df_of_csv_file.iloc[i]['дата время'].hour
            file.write(struct.pack('B', hour*8 + (minute//8))) # час и минута деленная на 8

            ##### блок написания даты
            month = df_of_csv_file.iloc[i]['дата время'].month
            day = df_of_csv_file.iloc[i]['дата время'].day  
            file.write(struct.pack('B', day+32*(month%8))) # дата от 01.01.1980
            ## Блок написания года 
            year = df_of_csv_file.iloc[i]['дата время'].year-1980
            polovina_goda=0
            if month >=8:
                polovina_goda=1
            file.write(struct.pack('B', year*2+polovina_goda))
            ###########################
            ###########################

            ### блок запосления номера косы  не могу правильность проверить
        #     file.write(b'\x64\x00') # , номер косы
            file.write(struct.pack('>h', df_of_csv_file.iloc[i]['коса']))
            ###
            file.write(b'\x00\x00') # , превышение уровня, у нас его нет

            ### блок названия объекта
        #     file.write(b'\x04HUU\x00') # название объекта - \x длинна названия+1, название, пустой байт
        #     name_object = str(df_of_raw_data_files.iloc[i]['скважина'])
        #     if len(name_object) <3:
        #         name_object = ' '*(3-len(name_object)) + name_object

        #     file.write(b'\x04') # длина имени +1
            file.write(struct.pack('B', len_of_well_name+1))
            file.write(bytes(name_object, 'utf-8'))
            file.write(b'\x00')
            ###

            ###блок написания температур
            actual_columns = ['0,0', '0,5', '1,0', '1,5', '2,0', '2,5', '3,0', '3,5', '4,0', 
                              '4,5', '5,0', '6,0', '7,0', '8,0', '9,0', '10,0', '12,0', '14,0', '15,0']
            offset_sensor = [0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 10, 10, 10, 10, 10, 20, 20, 10]
        #     actual_columns = ['0,0'] # временно
            for index, j in enumerate(actual_columns):
                if str(df_of_csv_file.iloc[i][j]) != 'nan': # проверка на nan
                    current_temperature_number = float(df_of_csv_file.iloc[i][j])
                    integer_part = int(abs(current_temperature_number)//1)*100
                    decimal_part = int(round(abs(current_temperature_number)*100, 0)%100)
                    number = integer_part+decimal_part
                    if current_temperature_number<0:
                        number = number*(-1)
                    file.write(struct.pack('>h', number))
                    file.write(b'\xfe\n') #пишем степень и знак цельсия
                    file.write(struct.pack('>B', offset_sensor[index]))

            ### последний байт, длинна строки
            file.write(struct.pack('>h', 96+len_of_well_name+more_sensors*5))
        file.close()
#     print (list_of_csv_path)
print('end')